## Introduction

This project will focus on comparing the top 200 most populous cities in the US based on the types of venues in the area.  The cities will then be clustered using the k-means algorithm.  The target audience for this project would be people who are looking to expand their businesses from one city to another.  Cities in the same cluster may have like-minded citizens and similar business needs.  For example, if cities A and B are in the same cluster, a business owner in city A may have reason to believe that his or her business would also succeed in city B.  People that are moving to a different city may also have interest in this project.  They could use the clusters to determine which new cities are most like the one where they currently live.

## Data

This project will utilize data from Foursquare to compile the most common types of venues in each of the cities.  It will also use US city population data from the internet and location data accessed with geopy.  For example, the 8th largest city in the US is San Diego, as shown on this web page https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population.  This table could be scraped into a pandas dataframe using the BeautifulSoup package.  The coordinates of San Diego are 32.8153, -117.1350, which could be added to the cities dataframe through the use of the geopy python package.  Using the location data in the dataframe, an API call could be made to Foursquare to retrieve the first 200 venues within a 1 mile radius of the given coordinates.  The venues would then be classified using one-hot encoding of the venue types.  This process would be repeated for the remaining 199 cities.  Once this process is finished, the top 10 venue types in each city would be compiled and entered into the k-means algorithm to generate the clusters.

## Methodology

Import the necessary modules

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from IPython.display import clear_output
from geopy.geocoders import Nominatim
import folium
from sklearn.cluster import KMeans
import re

Pull the city data from the internet

In [18]:
res = requests.get('https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population')
soup = BeautifulSoup(res.content, 'lxml')
table = soup.find_all('table')[4]
df = pd.read_html(str(table))[0]
df.head()

,2019rank,City,State[c],2019estimate,2010Census,Change,2016 land area,2016 land area.1,2016 population density,2016 population density.1,Location
0,1,New York[d],New York,8336817,8175133,+1.98%,301.5 sq mi,780.9 km2,"28,317/sq mi","10,933/km2",40°39′49″N 73°56′19″W﻿ / ﻿40.6635°N 73.9387°W
1,2,Los Angeles,California,3979576,3792621,+4.93%,468.7 sq mi,"1,213.9 km2","8,484/sq mi","3,276/km2",34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°W
2,3,Chicago,Illinois,2693976,2695598,−0.06%,227.3 sq mi,588.7 km2,"11,900/sq mi","4,600/km2",41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W
3,4,Houston[3],Texas,2320268,2100263,+10.48%,637.5 sq mi,"1,651.1 km2","3,613/sq mi","1,395/km2",29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W
4,5,Phoenix,Arizona,1680992,1445632,+16.28%,517.6 sq mi,"1,340.6 km2","3,120/sq mi","1,200/km2",33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°W


Format the city data into a cleaned dataframe

In [19]:
df = df[['2019rank', 'City', 'State[c]']]
df = df[:200]
df.columns = ['Rank','City','State']
fixed_cities = []
for index, data in df.iterrows():
    fixed_cities.append(re.sub(r'\[[^)]*\]', '', data['City']))
cities_df = pd.DataFrame(fixed_cities, columns=['City'])
df['City'] = cities_df['City']
df.head()

,Rank,City,State
0,1,New York,New York
1,2,Los Angeles,California
2,3,Chicago,Illinois
3,4,Houston,Texas
4,5,Phoenix,Arizona


Retrieve city location data from geopy

In [22]:
geolocator = Nominatim(user_agent='Coursera_Capstone')
longitudes = []
latitudes = []
for index, data in df.iterrows():
    location = geolocator.geocode(f'{data[1]},{data[2]}')
    longitudes.append(location.longitude)
    latitudes.append(location.latitude)

Add the location data to the data frame

In [24]:
df.insert(loc=3, column='Latitude', value=latitudes)
df.insert(loc=4, column='Longitude', value=longitudes)
df.head()

,Rank,City,State,Latitude,Longitude
0,1,New York,New York,40.712728,-74.006015
1,2,Los Angeles,California,34.053691,-118.242767
2,3,Chicago,Illinois,41.875562,-87.624421
3,4,Houston,Texas,29.758938,-95.367697
4,5,Phoenix,Arizona,33.448437,-112.074142


Define a function for pulling venues data from Foursquare

In [28]:
def get_venues(cities, states, latitudes, longitudes, radius=1600, limit=100):
    
    CLIENT_ID = str(input('Please enter your Foursquare client ID: '))
    CLIENT_SECRET = str(input('Please enter your Foursquare client secret: '))
    VERSION = str(input('Please enter the Foursquare version: '))
    clear_output()
    
    venues_list = []
    for city, lat, lng, state in zip(cities, latitudes, longitudes, states):
        
        url = f'''https://api.foursquare.com/v2/venues/explore?
        &client_id={CLIENT_ID}
        &client_secret={CLIENT_SECRET}
        &v={VERSION}
        &ll={lat},{lng}
        &radius={radius}
        &limit={limit}'''
        
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        venues_list.append([(
            city,
            lat,
            lng,
            state,
            venue['venue']['name'],
            venue['venue']['location']['lat'],
            venue['venue']['location']['lng'],
            venue['venue']['categories'][0]['name']) for venue in results])
    
    venues_df = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    venues_df.columns=['City',
                      'City Latitude',
                      'City Longitude',
                      'State',
                      'Venue Name',
                      'Venue Latitude',
                      'Venue Longitude',
                      'Venue Category']
    
    print('Venues retrieved successfully')
    return(venues_df)

Retrieve the venues data from Foursquare

In [29]:
city_venues = get_venues(cities=df['City'], states=df['State'], latitudes=df['Latitude'], longitudes=df['Longitude'])

Venues retrieved successfully


View the shape and first five rows of the venues dataframe

In [30]:
print(city_venues.shape)
city_venues.head()

(15750, 8)


,City,City Latitude,City Longitude,State,Venue Name,Venue Latitude,Venue Longitude,Venue Category
0,New York,40.712728,-74.006015,New York,The Bar Room at Temple Court,40.711448,-74.006802,Hotel Bar
1,New York,40.712728,-74.006015,New York,"The Beekman, A Thompson Hotel",40.711173,-74.006702,Hotel
2,New York,40.712728,-74.006015,New York,The Class by Taryn Toomey,40.712753,-74.008734,Gym / Fitness Center
3,New York,40.712728,-74.006015,New York,City Hall Park,40.712415,-74.006724,Park
4,New York,40.712728,-74.006015,New York,The Wooly Daily,40.712137,-74.008395,Coffee Shop


View the venue counts grouped by city

In [31]:
city_venues.groupby(['City','State']).count()

,,City Latitude,City Longitude,Venue Name,Venue Latitude,Venue Longitude,Venue Category
City,State,,,,,,
Akron,Ohio,91,91,91,91,91,91
Albuquerque,New Mexico,100,100,100,100,100,100
Alexandria,Virginia,100,100,100,100,100,100
Amarillo,Texas,35,35,35,35,35,35
Anaheim,California,64,64,64,64,64,64
...,...,...,...,...,...,...,...
Washington,District of Columbia,100,100,100,100,100,100
Wichita,Kansas,100,100,100,100,100,100
Winston–Salem,North Carolina,92,92,92,92,92,92


Create categorical variables for the venue categories using one-hot encoding

In [34]:
venues_onehot = pd.get_dummies(city_venues[['Venue Category']], prefix="", prefix_sep="")

venues_onehot.insert(0, column='City', value=city_venues['City'].tolist())
venues_onehot.insert(1, column='State', value=city_venues['State'].tolist())

venues_onehot.head()

,City,State,ATM,Accessories Store,Advertising Agency,Afghan Restaurant,African Restaurant,Airport,Airport Service,American Restaurant,...,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,New York,New York,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,New York,New York,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,New York,New York,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,New York,New York,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,New York,New York,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Group the venues by city and calculate the mean value for each category

In [56]:
venues_grouped = venues_onehot.groupby(['City','State']).mean().reset_index()
print(venues_grouped.shape)
venues_grouped

(199, 471)


,City,State,ATM,Accessories Store,Advertising Agency,Afghan Restaurant,African Restaurant,Airport,Airport Service,American Restaurant,...,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Akron,Ohio,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.032967,...,0.0,0.010989,0.010989,0.00,0.0,0.00,0.0,0.000000,0.00,0.0
1,Albuquerque,New Mexico,0.02,0.0,0.0,0.0,0.0,0.0,0.0,0.020000,...,0.0,0.000000,0.000000,0.00,0.0,0.00,0.0,0.000000,0.01,0.0
2,Alexandria,Virginia,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.040000,...,0.0,0.000000,0.000000,0.01,0.0,0.00,0.0,0.020000,0.00,0.0
3,Amarillo,Texas,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.057143,...,0.0,0.000000,0.028571,0.00,0.0,0.00,0.0,0.000000,0.00,0.0
4,Anaheim,California,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.015625,...,0.0,0.000000,0.000000,0.00,0.0,0.00,0.0,0.000000,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,Washington,District of Columbia,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.030000,...,0.0,0.000000,0.000000,0.00,0.0,0.00,0.0,0.010000,0.00,0.0
195,Wichita,Kansas,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.050000,...,0.0,0.000000,0.000000,0.01,0.0,0.00,0.0,0.000000,0.00,0.0
196,Winston–Salem,North Carolina,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.076087,...,0.0,0.000000,0.021739,0.00,0.0,0.00,0.0,0.021739,0.00,0.0
197,Worcester,Massachusetts,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.030000,...,0.0,0.010000,0.010000,0.00,0.0,0.01,0.0,0.000000,0.00,0.0


Define a function for finding the most common venue types for a given city

In [163]:
def most_common_venues(row, num_venues):
    categories = row.iloc[2:]
    categories_sorted = categories.sort_values(ascending=False)
    
    return categories_sorted.index.values[0:num_venues]

Calculate the 10 most common venue types for each city

In [164]:
num_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['City','State']
for ind in np.arange(num_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

city_venues_sorted = pd.DataFrame(columns=columns)
city_venues_sorted['City'] = venues_grouped['City']
city_venues_sorted['State'] = venues_grouped['State']

for ind in np.arange(venues_grouped.shape[0]):
    city_venues_sorted.iloc[ind, 2:] = most_common_venues(venues_grouped.iloc[ind, :], num_venues)

city_venues_sorted.head()

,City,State,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Akron,Ohio,Sandwich Place,Bar,Rental Car Location,Italian Restaurant,Coffee Shop,Music Venue,American Restaurant,Bank,Café,Fast Food Restaurant
1,Albuquerque,New Mexico,Coffee Shop,Pizza Place,Sandwich Place,Brewery,Hotel,Restaurant,Mexican Restaurant,Bar,New American Restaurant,Diner
2,Alexandria,Virginia,Park,Coffee Shop,Pizza Place,American Restaurant,Grocery Store,New American Restaurant,Seafood Restaurant,Spa,Hotel,Gastropub
3,Amarillo,Texas,Mexican Restaurant,Restaurant,Sandwich Place,Café,American Restaurant,Sushi Restaurant,Convenience Store,Performing Arts Venue,Furniture / Home Store,Fast Food Restaurant
4,Anaheim,California,Mexican Restaurant,Convenience Store,Taco Place,Coffee Shop,Indian Restaurant,Burger Joint,Brewery,Ice Cream Shop,Bakery,Steakhouse


Create the k-means object and fit it to the categorical data.  K-means was chosen for this project because it creates consistent results without too much over- or under-fitting.  DBSCAN was initially tested but it only resulted in 1-2 clusters or hundreds of clusters.  This is likely because the cities don't cluster densely enough to form regions that are distinguishable by DBSCAN.

In [165]:
kclusters = 15

venues_grouped_clustering = venues_grouped.drop(['City','State'], 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(venues_grouped_clustering)

labels = kmeans.labels_

Merge the cluster labels with the city venue data

In [166]:
city_venues_sorted.insert(0, 'Cluster Label', labels)

cities_merged = df

cities_merged = cities_merged.join(city_venues_sorted.set_index(['City','State']), on=['City','State'])
cities_merged.dropna(inplace=True)

cities_merged

,Rank,City,State,Latitude,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,New York,New York,40.712728,-74.006015,6.0,Park,Coffee Shop,Hotel,Wine Shop,Gym,Memorial Site,Spa,Café,Burger Joint,Men's Store
1,2,Los Angeles,California,34.053691,-118.242767,3.0,Ice Cream Shop,Japanese Restaurant,Music Venue,Bookstore,Ramen Restaurant,Plaza,Coffee Shop,Bar,Theater,Brewery
2,3,Chicago,Illinois,41.875562,-87.624421,6.0,Hotel,Park,Pizza Place,Theater,Coffee Shop,Aquarium,Yoga Studio,Music Venue,Hot Dog Joint,Grocery Store
3,4,Houston,Texas,29.758938,-95.367697,14.0,Hotel,Park,Bar,Coffee Shop,Burger Joint,Baseball Stadium,Mexican Restaurant,Steakhouse,Seafood Restaurant,New American Restaurant
4,5,Phoenix,Arizona,33.448437,-112.074142,9.0,Coffee Shop,American Restaurant,Art Gallery,Pizza Place,Lounge,Music Venue,Hotel,Cocktail Bar,Bar,Dessert Shop
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,196,Carrollton,Texas,32.953735,-96.890282,8.0,Fast Food Restaurant,Mexican Restaurant,Pizza Place,Fried Chicken Joint,Vietnamese Restaurant,Miscellaneous Shop,BBQ Joint,Grocery Store,Greek Restaurant,Sushi Restaurant
196,197,Waco,Texas,31.549333,-97.146670,3.0,Mexican Restaurant,American Restaurant,Home Service,Cajun / Creole Restaurant,Mediterranean Restaurant,Discount Store,Coffee Shop,BBQ Joint,Theater,Tea Room
197,198,Orange,California,33.750038,-117.870493,12.0,Mexican Restaurant,Fast Food Restaurant,Convenience Store,Bar,Coffee Shop,American Restaurant,Sandwich Place,Café,Restaurant,Pizza Place
198,199,Fullerton,California,33.870821,-117.929417,3.0,Sushi Restaurant,Pizza Place,Coffee Shop,Italian Restaurant,Burger Joint,Café,Mexican Restaurant,Sports Bar,Comic Shop,Cocktail Bar


Plot the cities on a map of the United States, colored by cluster number

In [174]:
geolocator = Nominatim(user_agent='Coursera_Capstone')
location = geolocator.geocode('United States')
latitude = location.latitude
longitude = location.longitude

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=4)

colors = ['#2f4f4f','#6b8e23','#a0522d','#000080','#ff0000','#00ced1','#ffa500','#ffff00','#00ffff','#00ff00','#00fa9a','#0000ff','#d8bfd8','#ff00ff','#1e90ff']

for lat, lon, poi, cluster in zip(cities_merged['Latitude'], cities_merged['Longitude'], cities_merged['City'], cities_merged['Cluster Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=colors[int(cluster-1)],
        fill=True,
        fill_color=colors[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Count the number of cities in each cluster

In [235]:
cluster_counts = cities_merged.groupby('Cluster Label').count()[['Rank']]
cluster_counts.columns = ['Count']
cluster_counts

,Count
Cluster Label,
0.0,3
1.0,21
2.0,11
3.0,40
4.0,5
5.0,1
6.0,22
7.0,1
8.0,7


Create a dictionary of dataframes, each consisting of cities from one cluster

In [248]:
clusters_dict = {}

for cluster in range(kclusters):
    clusters_dict[f'cluster_{cluster}'] = cities_merged.loc[cities_merged['Cluster Label'] == cluster, cities_merged.columns[[1] + [2] + list(range(5, cities_merged.shape[1]))]]

Create a dictionary of the most common venue categories in each cluster

In [249]:
cluster_cats = {}

for key, value in clusters_dict.items():
    prefixes = ['1st','2nd','3rd']
    cluster_cats[key] = {}
    for prefix in prefixes:
        cluster_cats[key][prefix] = list(value.groupby(f'{prefix} Most Common Venue').count().sort_values(by='City', ascending=False).iloc[0:3]['City'].index)

## Results and Discussion

Through analysis of venue categories and clustering of cities, the top 200 most populous cities in the United States have been grouped into fifteen different clusters.  These clusters were created based on similiarites in the types of venues in a given area.  Through visualizing all of the clusters on a map, one can see that these clusters also frequently share geographic similarities as well.  Take for example cluster 12, which tends to be centered around the southwestern portion of the country.  The cities within cluster 12 are displayed below:

In [256]:
clusters_dict['cluster_12'][['City','State']].sort_values(by='State')

,City,State
87,Glendale,Arizona
34,Mesa,Arizona
146,Peoria,Arizona
80,Chandler,Arizona
162,Lancaster,California
136,Ontario,California
163,Salinas,California
119,Huntington Beach,California
170,Pomona,California
109,Oxnard,California


The vast majority of the cities within this cluster are located within California, Texas, and Arizona.  In addition to creating the fifteen clusters of cities, a dictionary was created that contains the most common venue types for each cluster.  Continuing with the example of cluster 12, the most common venue types are as follows:

In [254]:
print(f'First most common venue types: {cluster_cats["cluster_12"]["1st"]}')
print(f'Second most common venue types: {cluster_cats["cluster_12"]["2nd"]}')
print(f'Third most common venue types: {cluster_cats["cluster_12"]["3rd"]}')

First most common venue types: ['Mexican Restaurant', 'Fast Food Restaurant', 'Burger Joint']
Second most common venue types: ['Convenience Store', 'Mexican Restaurant', 'Coffee Shop']
Third most common venue types: ['Fast Food Restaurant', 'Pizza Place', 'Sandwich Place']


We can see that some of the most common venue types are Mexican restaurants.  This makes sense for the southwestern United States, as these states are close to the border with Mexico and have significant immigrant populations.

This project could be a powerful tool as a starting point for someone who is looking to open a business in a new city or move to a new city.  This can be seen in the cluster 12 example.  Someone who has lived in Chula Vista, California for a long time would likely want to move somewhere new that also has a high density of Mexican restaurants.  Mesa, Arizona or El Paso, Texas could be good places to start a search.  When moving to a new city, familiar venues could help someone to feel more at home.

One possible weakness of this project is it strongly biases larger cities.  For example, no cities from Wyoming, Montana, or North Dakota are included in this analysis.  Smaller cities would be part of distinct clusters as they likely have different venue distributions that large cities.  This could be remedied by pulling additionaly city data from other webpages and training the model again.  This would not require too much effort, as it is as simple as changing the url in the request line or copying the cell and concatonating the new table with the existing one.

One other interesting feature of this analysis to note is that Foursquare did not retrieve any venue data for San Bernardino, California.  This was remedied by using the pd.dropna() function to remove the missing data from the model but could be due to faulty location data from geopy.  In future versions, location verification steps could be implemented to ensure that no cities are missed.

## Conclusion

The goal of this project was to create a clustering model to group together the top 200 most populous cities in the United States based on the most common venue categories in each city.  City data was firt pulled from Wikipedia through the use of the BeautifulSoup package.  Geographical data for each city was then retrieved using the geopy package and concatonated to the existing cities dataframe.  To get venue data, API calls were made to Foursquare using the previously obtained location data.  In this analysis, the venue limit was set to 100 venues and the radius was set to one mile.  The venue data was then transformed to categorial data using one-hot encoding and all of the venues were grouped together by city.  To get an idea of the types of venues in each city, the mean values for each venue category were calculated.  These mean values were then sorted to get the top ten venue categories for each city.  After this, all of the city category data was put into a k-means clustering algorithm.  This algorithm returned 15 distinct clusters of cities.  These clusters are based off of most common venue types but also happen to share some geographical similarities, as seen on the folium map visualization.  Finally, a dictionary was created containing the most common venue types for categories for each cluster.

This project is primarily useful as a starting point for people searching for new cities that are similar to the one in which they currently live.  After getting an inital list of cities, they should still do more research into each of the cities, as the most common types of venues are not all that determines what it is like to live in a city or how well a business will do in a new city.